# Predicting digits using MNIST Dataset

I'm using my knowledge of TensorFlow to create a basic model of neural network from scratch.
This notebook uses one of the sample dataset in TensorFlow libraries, MNIST dataset, which contains images of handwritten digits.

I've got 95.14% accuracy on this by using basic model using 3 hidden layers, softmax_crossentropy loss function and AdamOptimizer.


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

The input data that was imported consists of three parts. Train, Test and Validation. 

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

Let's fix number of nodes for each layer in our neural network.

In [19]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

Also, let's look at the number of classes(0..9) to be classified into and also the batch size.

In [10]:
n_classes = 10
batch_size = 100

I'm using some TensorFlow placeholders to use them later as parameters. Dataset is 28*28, after Flatten, we get internally dimension of 784 length.

In [12]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float")

Let's use the core concept of deep learning, computations are done by setting weights for each connection of one in a layer to the other in the next layer. A bias term is added to ensure that the result is not always zero whenever data or weights is zero. If the result from a layer is all zero for certain entries, then it would be biased for that layer as the contribution of the following layers is very insignificant. 

This is the computation to be done for each layer.
`(weights * data) + biases`

Let's use random_normal function for generating random values. We use ReLU activation function on each layer.

Feedforward model as the output of one layer is given as input to the next layer.

In [20]:
def neural_network_model(data):
    hiddenlayer_1 = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hiddenlayer_2 = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hiddenlayer_3 = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])), 
                        'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data,hiddenlayer_1['weights']), hiddenlayer_1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1,hiddenlayer_2['weights']), hiddenlayer_2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hiddenlayer_3['weights']), hiddenlayer_3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']), output_layer['biases'])
    return output

Now, given that our model is implemented, let's train over training data.

The cost function we consider here is reduced mean of the softmax cross entropy results over the labels.

So, now that done, we go into running our cycles. Each cycle (epoch) consists of a feedforward stride and a backpropagtion result used to improve the weights internally. Also for each epoch, we calculate corresponding loss after using AdamOptimizer to reduce minimize the cost function. We'll display the progress.

By comparing which predictions actual match, accuracy is calculated by reduced mean. Now, let's evaluate over the completely unknown test data.

In [21]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    n_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(n_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                x_, y_ = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: x_, y: y_})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of',n_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

Let's run this.

In [22]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch 0 completed out of 10 loss: 1981399.3802947998
Epoch 1 completed out of 10 loss: 406695.8919067383
Epoch 2 completed out of 10 loss: 221790.25764799118
Epoch 3 completed out of 10 loss: 131552.14850997925
Epoch 4 completed out of 10 loss: 77057.46531385183
Epoch 5 completed out of 10 loss: 51720.33450707793
Epoch 6 completed out of 10 loss: 34879.249119613785
Epoch 7 completed out of 10 loss: 26638.419391626492
Epoch 8 completed out of 10 loss: 21474.561464890838
Epoch 9 completed out of 10 loss: 20062.26769438386
Accuracy: 0.9514
